In [1]:
import requests
import re
from bs4 import BeautifulSoup
import math
import pandas as pd
import time
import random
from IPython.display import clear_output
import wrangle

https://www.scrapingdog.com/blog/scrape-linkedin-jobs/

https://www.geeksforgeeks.org/using-jupyter-notebook-in-virtual-environment/

# Target URL
Follow the steps to produce a target URL:
1. Goto Linkedin and search for a job with any filters you want.
2. Open up the Developer inspector (Use Google Chrome) and scroll down until the next set of jobs load.
3. In the network tab there should be a "search" request when it loads the new jobs.
4. Grab that URL in the search request.
5. replace the last NUMBER in the URL with a {} in order to utilize a .format() for looping later. ie 'start=0' -> 'start={}'

In [2]:
target_url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%2BScientist&location=United%2BStates&geoId=103644278&f_TPR=r86400&f_JT=F&f_E=2%2C4&f_WT=2&currentJobId=3692555868&position=2&pageNum=2&start={}'

# Job ID Loop
Used to get the Job ID to appened to the job URL for crawling

In [3]:
job_ids_list = wrangle.job_id_scrape(target_url, 1000)

In [4]:
len(job_ids_list)

973

# Job Loop
This is made outside of a function on purpose because if the function fails or is interupted at any step it won't return a DF.

In [16]:

df = {'job_title' : [], 'company' : [], 'job_info' : [], 'when_posted' : [], 'num_applicants' : [],
      'seniority' : [], 'employment_type' : [], 'job_function' : [], 'industry' : [], 'job_url' : []}
target_job_url = 'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}/' #format of the URL. The {} is where the jobID gets formated to.
for i in range(0, len(job_ids_list)): # Main loop
    res_job = requests.get(target_job_url.format(job_ids_list[i]))
    if res_job.status_code != 200:
        print('Status code not 200. Sleeping for 1 minute')
        time.sleep(60) #Long sleep in case of IP block
    else:
        soup = BeautifulSoup(res_job.text, 'html.parser')
        try:
            df['job_title'].append(soup.find('div',{'class':'top-card-layout__entity-info'}).find('a').text.strip()) #Title
        except:
            df['job_title'].append(None)
        try:
            df['company'].append(soup.find('div',{'class':'top-card-layout__card'}).find('a').find('img').get('alt')) #Company
        except:
            df['company'].append(None)
        try:
            df['job_info'].append(soup.find('div',{'class':'show-more-less-html__markup'}).text.strip()) #Bulk, might need to find a way to clean this up.
        except:
            df['job_info'].append(None)
        try:
            df['when_posted'].append(soup.find('span',{'class':'posted-time-ago__text'}).text.replace('\n', '').strip()) #When was it posted
        except:
            df['when_posted'].append(None)
        num_of_apps1 = soup.find('figcaption',{'class':'num-applicants__caption'})  #number of applicants scape attempt1
        if num_of_apps1 == None: #Check first soup, if it fails check second soup
            num_of_apps2 = soup.find('span', class_='num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet') #number of applicants scrape attempt2
            if num_of_apps2 == None: #Check second soup, if it fails, append None
                df['num_applicants'].append(None) #if nosoup works just append None
            else:
                df['num_applicants'].append(num_of_apps2.text.replace('\n', '').strip()) #If first soup failed and second soup passed, append second soup
        else:
            df['num_applicants'].append(num_of_apps1.text.replace('\n', '').strip()) #If first soup works, append first soup
        job_attributes = soup.find('ul',{'class':'description__job-criteria-list'}) #Job attributes
        if job_attributes == None: #Check to see if there is soup. If not, append none.
            df['seniority'].append(None)
            df['employment_type'].append(None)
            df['job_function'].append(None)
            df['industry'].append(None)
        else:
            job_attributes = job_attributes.text.replace('\n', '').strip()
            if 'Seniority level' in job_attributes:
                pattern = 'Seniority level\s+(?P<seniority>(.*?)(?:\s{2,}|$))'
                df['seniority'].append(re.search(pattern, job_attributes).groups()[0])
            else:
                df['seniority'].append(None)
            if 'Employment type' in job_attributes:
                pattern = 'Employment type\s+(?P<employment_type>(.*?)(?:\s{2,}|$))'
                df['employment_type'].append(re.search(pattern, job_attributes).groups()[0])
            else:
                df['employment_type'].append(None)
            if 'Job function' in job_attributes:
                pattern = 'Job function\s+(?P<job_function>(.*?)(?:\s{2,}|$))'
                df['job_function'].append(re.search(pattern, job_attributes).groups()[0])
            else:
                df['job_function'].append(None)
            if 'Industries' in job_attributes:
                pattern = 'Industries\s+(?P<industries>(.*?)(?:\s{2,}|$))'
                df['industry'].append(re.search(pattern, job_attributes).groups()[0])
            else:
                df['industry'].append(None)
        clear_output()
        print(f'Obatined info for id: {job_ids_list[i]}')
        df['job_url'].append(f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_ids_list[0]}/')
        job_ids_list.pop(0)
        time.sleep(random.choice(range(2,4,1))) # Sleep to avoid IP block

Obatined info for id: 3696204960


AttributeError: 'NoneType' object has no attribute 'groups'

In [ ]:
df = pd.DataFrame(df)
df.head()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        5 non-null      object
 1   company          5 non-null      object
 2   job_info         5 non-null      object
 3   when_posted      5 non-null      object
 4   num_applicants   5 non-null      object
 5   seniority        5 non-null      object
 6   employment_type  5 non-null      object
 7   job_function     5 non-null      object
 8   industry         5 non-null      object
 9   job_id           5 non-null      object
dtypes: object(10)
memory usage: 532.0+ bytes


In [30]:
def job_scrape(job_dict: dict, job_id_list: list) -> tuple[dict, list]:
    if job_dict == {}:
        job_dict = {'job_title' : [], 'company' : [], 'job_info' : [], 'when_posted' : [], 'num_applicants' : [],
      'seniority' : [], 'employment_type' : [], 'job_function' : [], 'industry' : [], 'job_url' : []}
    else:
        pass
    target_job_url = 'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}/' #format of the URL. The {} is where the jobID gets formated to.
    res_job = requests.get(target_job_url.format(job_ids_list[0]))
    if res_job.status_code != 200:
        print('Status code not 200. Sleeping for 1 minute')
        time.sleep(60) #Long sleep in case of IP block
    else:
        soup = BeautifulSoup(res_job.text, 'html.parser')
        try:
            job_dict['job_title'].append(soup.find('div',{'class':'top-card-layout__entity-info'}).find('a').text.strip()) #Title
        except:
            job_dict['job_title'].append(None)
        try:
            job_dict['company'].append(soup.find('div',{'class':'top-card-layout__card'}).find('a').find('img').get('alt')) #Company
        except:
            job_dict['company'].append(None)
        try:
            job_dict['job_info'].append(soup.find('div',{'class':'show-more-less-html__markup'}).text.strip()) #Bulk, might need to find a way to clean this up.
        except:
            job_dict['job_info'].append(None)
        try:
            job_dict['when_posted'].append(soup.find('span',{'class':'posted-time-ago__text'}).text.replace('\n', '').strip()) #When was it posted
        except:
            job_dict['when_posted'].append(None)
        num_of_apps1 = soup.find('figcaption',{'class':'num-applicants__caption'})  #number of applicants scape attempt1
        if num_of_apps1 == None: #Check first soup, if it fails check second soup
            num_of_apps2 = soup.find('span', class_='num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet') #number of applicants scrape attempt2
            if num_of_apps2 == None: #Check second soup, if it fails, append None
                job_dict['num_applicants'].append(None) #if nosoup works just append None
            else:
                job_dict['num_applicants'].append(num_of_apps2.text.replace('\n', '').strip()) #If first soup failed and second soup passed, append second soup
        else:
            job_dict['num_applicants'].append(num_of_apps1.text.replace('\n', '').strip()) #If first soup works, append first soup
        job_attributes = soup.find('ul',{'class':'description__job-criteria-list'}) #Job attributes
        if job_attributes == None: #Check to see if there is soup. If not, append none.
            job_dict['seniority'].append(None)
            job_dict['employment_type'].append(None)
            job_dict['job_function'].append(None)
            job_dict['industry'].append(None)
        else:
            job_attributes = job_attributes.text.replace('\n', '').strip()
            if 'Seniority level' in job_attributes:
                pattern = 'Seniority level\s+(?P<seniority>(.*?)(?:\s{2,}|$))'
                job_dict['seniority'].append(re.search(pattern, job_attributes).groups()[0])
            else:
                job_dict['seniority'].append(None)
            if 'Employment type' in job_attributes:
                pattern = 'Employment type\s+(?P<employment_type>(.*?)(?:\s{2,}|$))'
                job_dict['employment_type'].append(re.search(pattern, job_attributes).groups()[0])
            else:
                job_dict['employment_type'].append(None)
            if 'Job function' in job_attributes:
                pattern = 'Job function\s+(?P<job_function>(.*?)(?:\s{2,}|$))'
                job_dict['job_function'].append(re.search(pattern, job_attributes).groups()[0])
            else:
                job_dict['job_function'].append(None)
            if 'Industries' in job_attributes:
                pattern = 'Industries\s+(?P<industries>(.*?)(?:\s{2,}|$))'
                job_dict['industry'].append(re.search(pattern, job_attributes).groups()[0])
            else:
                job_dict['industry'].append(None)
        job_dict['job_url'].append(f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_ids_list[0]}/')
        job_ids_list.pop(0)
        time.sleep(random.choice(range(2,4,1))) # Sleep to avoid IP block
    return job_dict, job_id_list

In [38]:
len(job_ids_list)

950

In [41]:
#job_data = {}
for i in range(0, len(job_ids_list)):
    job_data, job_ids_list = job_scrape(job_data, job_ids_list)

KeyboardInterrupt: 

In [42]:
pd.DataFrame(job_data)

,job_title,company,job_info,when_posted,num_applicants,seniority,employment_type,job_function,industry,job_url
0,Senior Data Scientist,Harnham,Senior Data ScientistHealthcare | United State...,4 hours ago,53 applicants,Mid-Senior level,Full-time,Information Technology and Health Care Provide...,Hospitals and Health Care,https://www.linkedin.com/jobs-guest/jobs/api/j...
1,Python/R Developer,Sonitalent Corp,Role: Python/R DeveloperLocation:100% remote(W...,23 hours ago,Over 200 applicants,Entry level,Full-time,Engineering and Information Technology ...,Staffing and Recruiting,https://www.linkedin.com/jobs-guest/jobs/api/j...
2,Senior Data Scientist,"Qventus, Inc",The CompanyHave you ever found yourself or a l...,13 hours ago,84 applicants,Mid-Senior level,Full-time,Engineering and Information Technology ...,Hospitals and Health Care,https://www.linkedin.com/jobs-guest/jobs/api/j...
3,Staff Data Scientist,"Machinify, Inc.",Machinify is a revolutionary healthcare softwa...,20 hours ago,Be among the first 25 applicants,Entry level,Full-time,Engineering and Information Technology ...,Software Development,https://www.linkedin.com/jobs-guest/jobs/api/j...
4,Python Developer - Remote | WFH,Get It Recruit - Information Technology,Are you ready to embark on a journey with a pi...,15 hours ago,48 applicants,Entry level,Full-time,Engineering and Information Technology ...,Human Resources Services,https://www.linkedin.com/jobs-guest/jobs/api/j...
5,Data Scientist,ManTech,"Secure our Nation, Ignite your FutureData Scie...",6 hours ago,28 applicants,Entry level,Full-time,Engineering and Information Technology ...,IT Services and IT Consulting,https://www.linkedin.com/jobs-guest/jobs/api/j...
6,"Senior Clinical Data Scientist, Real-World Dat...",Freenome,Why join Freenome?Freenome is a high-growth bi...,12 hours ago,36 applicants,Mid-Senior level,Full-time,Engineering and Information Technology ...,Biotechnology Research,https://www.linkedin.com/jobs-guest/jobs/api/j...


In [20]:
df_test = pd.DataFrame()

In [21]:
df_test

""


In [23]:
df_test.assign(test = 1)

,test


In [24]:
df_test

""
